# LangChain: Memory


In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
llm_model = "gpt4-turbo-default"

In [3]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [5]:
llm = AzureChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

/home/snandakumar/anaconda3/envs/LLM/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [6]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's great to meet you. How's your day going so far?"

In [7]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's great to meet you. How's your day going so far?
Human: What is 1+1?
AI:

> Finished chain.


"1+1 equals 2. It's a basic arithmetic operation. Is there anything else you'd like to know or talk about?"

In [8]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's great to meet you. How's your day going so far?
Human: What is 1+1?
AI: 1+1 equals 2. It's a basic arithmetic operation. Is there anything else you'd like to know or talk about?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew. You mentioned it earlier in our conversation. What else would you like to discuss or ask about?'

In [9]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's great to meet you. How's your day going so far?
Human: What is 1+1?
AI: 1+1 equals 2. It's a basic arithmetic operation. Is there anything else you'd like to know or talk about?
Human: What is my name?
AI: Your name is Andrew. You mentioned it earlier in our conversation. What else would you like to discuss or ask about?


In [10]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's great to meet you. How's your day going so far?\nHuman: What is 1+1?\nAI: 1+1 equals 2. It's a basic arithmetic operation. Is there anything else you'd like to know or talk about?\nHuman: What is my name?\nAI: Your name is Andrew. You mentioned it earlier in our conversation. What else would you like to discuss or ask about?"}

In [13]:
memory = ConversationBufferMemory() ##we can set values in memory 

In [14]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [15]:
print(memory.buffer)

Human: Hi
AI: What's up


In [16]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [17]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [18]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

In [19]:
print(memory.buffer)

Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool


## ConversationBufferWindowMemory
The normal memory can overflow - It only uses the last K interactions. 

In [20]:
from langchain.memory import ConversationBufferWindowMemory

In [21]:
memory = ConversationBufferWindowMemory(k=1)               

In [22]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [23]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [26]:
llm = AzureChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [27]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's great to meet you. How's your day going so far?"

In [28]:
conversation.predict(input="What is 1+1?")

"1+1 equals 2. It's a basic arithmetic operation. Is there anything else you'd like to know or talk about?"

In [29]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information such as your name. Is there anything else you'd like to discuss or any other questions you have?"

## ConversationTokenBufferMemory

In [4]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import AzureOpenAI
llm = AzureChatOpenAI(temperature=0.0, model=llm_model)

/home/snandakumar/anaconda3/envs/LLM/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [5]:
#pip install --upgrade tiktoken

In [7]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=500)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model cl100k_base.See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.

## ConversationSummaryMemory

In [8]:
from langchain.memory import ConversationSummaryBufferMemory

In [9]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model cl100k_base.See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.